In [1]:
%%writefile install_packages.py

def install_packages():
  import os
  os.system('pip uninstall -y torch')
  os.system('pip uninstall -y torchvision')
  os.system('pip install -q --no-index --find-links=/kaggle/input/0-6-3-post1-wheels-vllm vllm')
  os.system('pip install -q --no-index -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl')
  os.system('pip install -q --no-index -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl')
  os.system('pip uninstall -y pynvml')
  os.system('pip install --no-deps --no-index /kaggle/input/0-6-3-post1-wheels-vllm/nvidia_ml_py-12.560.30-py3-none-any.whl')
  os.system('pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl')

install_packages()

Writing install_packages.py


In [2]:
try:
    import vllm
    print('vllm version=',vllm.__version__)
except ImportError:
    print('Installing packages')
    !python install_packages.py
    import vllm
    print('----'*10,'\nvllm version=',vllm.__version__)

Installing packages
Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: pynvml 11.4.1
Uninstalling pynvml-11.4.1:
  Successfully uninstalled pynvml-11.4.1
Processing /kaggle/input/0-6-3-post1-wheels-vllm/nvidia_ml_py-12.560.30-py3-none-any.whl
  Attempting uninstall: nvidia-ml-py
    Found existing installation: nvidia-ml-py 11.495.46
    Uninstalling nvidia-ml-py-11.495.46:
      Successfully uninstalled nvidia-ml-py-11.495.46
Processing /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl
---------------------------------------- 
vllm version= 0.6.3.post1


In [3]:
import pandas as pd
import numpy as np
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

run_eval = True ## To check score on train data- 100 samples

In [4]:
%%writefile run_vllm.py
system_prompt='''You are a skilled judge evaluating responses from two large language models(LLMs). Your task is to select the response that best meets the user's needs based on the query provided.

**Input Format:**
<Query>
[User's original query to both LLMs]
</Query>

<Response_A>
[First LLM's response]
</Response_A>

<Response_B>
[Second LLM's response]
</Response_B>

**Your Task:**
Carefully analyze both <Response_A> and <Response_B> in relation to the Query. Determine which response is more likely to be selected by a user based on the following criteria:
- Completeness in addressing the query
- Accuracy of information
- Clarity and coherence
- Conciseness vs appropriate detail
- Helpful examples or explanations when needed
- Professional yet engaging tone
- Sound reasoning and logic
- Format and presentation

'''

system_prompt_chatformat='''system
You are a skilled judge evaluating responses from two large language models(LLMs). Your task is to select the response that best meets the user's needs based on the query provided.

**Input Format:**
<Query>
[User's original query to both LLMs]
</Query>

<Response_A>
[First LLM's response]
</Response_A>

<Response_B>
[Second LLM's response]
</Response_B>

**Your Task:**
Carefully analyze both <Response_A> and <Response_B> in relation to the Query. Determine which response is more likely to be selected by a user based on the following criteria:
- Completeness in addressing the query
- Accuracy of information
- Clarity and coherence
- Conciseness vs appropriate detail
- Helpful examples or explanations when needed
- Professional yet engaging tone
- Sound reasoning and logic
- Format and presentation

'''

import sys
import re
import gc
import vllm
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModel
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor

def count_numbers_in_text(text):
    """Count the number of digits/numbers in a text string"""
    return len(re.findall(r'\d+', text))

def winner_with_postprocessing(logprob_dict, row, a_tok_id, b_tok_id, threshold=0.05):
    """Determine winner with post-processing logic"""
    try:
        if a_tok_id not in logprob_dict or b_tok_id not in logprob_dict:
            return 'A' if a_tok_id in logprob_dict else 'B'
            
        a_logit = logprob_dict[a_tok_id].logprob
        b_logit = logprob_dict[b_tok_id].logprob
        
        # If difference is smaller than threshold, count numbers
        if abs(a_logit - b_logit) < threshold:
            numbers_in_a = count_numbers_in_text(row['response_a'])
            numbers_in_b = count_numbers_in_text(row['response_b'])
            
            if numbers_in_a != numbers_in_b:
                return 'A' if numbers_in_a > numbers_in_b else 'B'
            else:
                # If number counts are equal, return based on logprobs
                return 'A' if a_logit > b_logit else 'B'
        else:
            # If difference is larger than threshold, use logprobs directly
            return 'A' if a_logit > b_logit else 'B'
    except:
        return 'A'  # Default fallback

IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

if IS_SUBMISSION:
    df = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet')
else:
    df = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet')
    df = df.sample(100, random_state=42).reset_index(drop=True).copy()
    df['winner_GT'] = df['winner']
    print('Length of df=',len(df))

def preprocess_function_truncation(row, tokenizer,max_length,system_prompt,system_prompt_chatformat):
    dot_tokens = tokenizer("......", add_special_tokens=False)["input_ids"]
    system_token= [tokenizer.bos_token_id]+tokenizer(system_prompt_chatformat,add_special_tokens=False) ["input_ids"]+[tokenizer.eos_token_id]
    user_token=tokenizer("user\n", add_special_tokens=False)["input_ids"]
    final_p_tokens = tokenizer('Which response is more likely to be selected by a user? (A or B)\n', add_special_tokens=False)["input_ids"]
    p=row['prompt']
    ra= row['response_a']
    rb =row['response_b']
    one_input_ids =system_token
    prev_tokens_num =  len(system_token)+len(final_p_tokens)+len(user_token)
    prompt=f'''**Here is your input to process now-**\nInput:\n\n<Query>\n{p}'''
    response_a=f'''\n<Response_A>\n{ra}'''
    response_b=f'''\n<Response_B>\n{rb}'''
    p_tokens  = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    ra_tokens = tokenizer(response_a, add_special_tokens=False)["input_ids"]
    rb_tokens = tokenizer(response_b, add_special_tokens=False)["input_ids"]
    a_end_token=tokenizer(f"""\n</Response_A>\n{'---'*10}\n""", add_special_tokens=False)["input_ids"]
    b_end_token=tokenizer(f"""\n</Response_B>\n\n""", add_special_tokens=False)["input_ids"]
    p_end_token=tokenizer(f"""\n</Query>\n{'---'*10}\n""", add_special_tokens=False)["input_ids"]
    all_tokens_num = prev_tokens_num +  len(p_tokens) + len(ra_tokens) + len(rb_tokens)+len(a_end_token) + len(b_end_token) + len(p_end_token)
    input_ids =[]
    if all_tokens_num > max_length:
        remain_tokens_num = max_length - prev_tokens_num  - 3*len(dot_tokens) 
        if remain_tokens_num >5:
            p_tokens  =  p_tokens[:int(remain_tokens_num*0.15)] + dot_tokens+ p_tokens[-int(remain_tokens_num*0.05):] if len( p_tokens) > int(remain_tokens_num*0.2) else  p_tokens
            ra_tokens = ra_tokens[:int(remain_tokens_num*0.3)] + dot_tokens+ ra_tokens[-int(remain_tokens_num*0.1):] if len(ra_tokens) > int(remain_tokens_num*0.4) else ra_tokens
            rb_tokens = rb_tokens[:int(remain_tokens_num*0.3)] + dot_tokens+ rb_tokens[-int(remain_tokens_num*0.1):] if len(rb_tokens) > int(remain_tokens_num*0.4) else rb_tokens
            input_ids = p_tokens+p_end_token + ra_tokens +a_end_token+ rb_tokens+b_end_token
    else:
        prev_tokens_num = all_tokens_num
        input_ids = p_tokens+p_end_token + ra_tokens +a_end_token+ rb_tokens+b_end_token
    input_ids += final_p_tokens
    text=tokenizer.decode(input_ids, skip_special_tokens=False)
    conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content":text},
        ]
    final_prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)+'Answer:\n'
    return final_prompt

def apply_template(row, tokenizer):
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user", 
            "content": f'''Here is your input to process now-

<Query>
{row['prompt']}
</Query>
{'---'*10}
<Response_A>
{row['response_a'][:6500]}
</Response_A>
{'---'*10}
<Response_B>
{row['response_b'][:6500]}
</Response_B>

Which response is more likely to be selected by a user? (Give the answer and though)
Output:
'''
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

model_path = "/kaggle/input/qwen2.5_14b_final/transformers/14b/1"

offload = 4
swap = 1
max_len = 12000
num_seqs = 256

tokenizer = AutoTokenizer.from_pretrained(model_path)

df["text"] = df.apply(lambda row: preprocess_function_truncation(row, tokenizer,max_length=4096,system_prompt=system_prompt,system_prompt_chatformat=system_prompt_chatformat),axis=1)

def tok_len(txt):
    return len(tokenizer.encode(txt))

df['token_length'] = df['text'].apply(tok_len)
print("max_len",max(df['token_length']))
print("Stats of df",df['token_length'].describe())
print('Example input-\n',df["text"][0])

llm = vllm.LLM(model=model_path,
    tensor_parallel_size=2,
    gpu_memory_utilization=0.99, 
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=max_len,
    disable_log_stats=True,
    cpu_offload_gb=offload,
    swap_space=swap,
    device='cuda',
    max_num_seqs=num_seqs,
    enable_prefix_caching=True,
)

responses = llm.generate(
    df["text"].values,
    vllm.SamplingParams(
        n=1,  
        top_k=1,  
        temperature=0,  
        skip_special_tokens=False, 
        max_tokens=1, 
        logprobs=20,
    ),
    use_tqdm=True
)

a_tok_id = tokenizer("A", add_special_tokens=False)["input_ids"][-1]
b_tok_id = tokenizer("B", add_special_tokens=False)["input_ids"][-1]

print(f">> EediRanker: A token id: {a_tok_id}")
print(f">> EediRanker: B token id: {b_tok_id}")

result = []
n = 0
for idx, response in enumerate(responses):
    try:
        logprob_dict = response.outputs[0].logprobs[0]
        n += 1
        top_tok_ids = set(list(logprob_dict.keys()))
        
        if len(top_tok_ids.intersection(set([a_tok_id, b_tok_id]))) == 0:
            print(f"Bad Output for {n}")
            result.append('A')
            continue
            
        # Use the new winner determination function
        winner_choice = winner_with_postprocessing(
            logprob_dict,
            df.iloc[idx],
            a_tok_id,
            b_tok_id,
            threshold=0.005
        )
        result.append(winner_choice)
        
    except Exception as e:
        print(f"Error processing response {n}: {str(e)}")
        result.append('A')

print('Raw responses: ', result)
df["winner"] = [f'model_{x.lower()}' for x in result]

if IS_SUBMISSION:
    df.to_csv("submission.csv", columns=["id", "winner"], index=False)
else:
    df.to_csv("submission.csv", columns=["id", "winner", "winner_GT"], index=False)

Writing run_vllm.py


In [5]:
# if IS_SUBMISSION or run_eval:
!python run_vllm.py

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Length of df= 100
max_len 4002
Stats of df count     100.000000
mean     1391.960000
std       918.510354
min       284.000000
25%       755.000000
50%      1188.000000
75%      1793.750000
max      4002.000000
Name: token_length, dtype: float64
Example input-
 <|im_start|>system
You are a skilled judge evaluating responses from two large language models(LLMs). Your task is to select the response that best meets the user's needs based on the query provided.

**Input Format:**
<Query>
[User's original query to both LLMs]
</Query>

<Response_A>
[First LLM's response]
</Response_A>

<Response_B>
[Second LLM's response]
</Response_B>

**Your Task:**
Carefully analyze both <Response_A> and <Response_B> in relation to the Query. Determine which response is more likely to be selected by a user based on the following criteria:
- Completeness in addressing the query
- Accuracy of information
- Clarity and coherence
- Conciseness vs appropriate detail
- Helpful examples or explanations when need

In [6]:
if not IS_SUBMISSION and run_eval:
    df = pd.read_csv('submission.csv')
    correct_preds = (df['winner'] == df['winner_GT']).sum()
    total_preds = len(df)
    acc = correct_preds / total_preds
    
    print(f"Accuracy: {acc}")

Accuracy: 0.85
